In [2]:
!pip install -U duckduckgo-search


   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 929.6 kB/s eta 0:00:04
   ----- ---------------------------------- 0.5/4.0 MB 929.6 kB/s eta 0:00:04
   ------- -------------------------------- 0.8/4.0 MB 838.9 kB/s eta 0:00:04
   ---------- ----------------------------- 1.0/4.0 MB 931.8 kB/s eta 0:00:04
   ------------- -------------------------- 1.3/4.0 MB 1.0 MB/s eta 0:00:03
   ------------- -------------------------- 1.3/4.0 MB 1.0 MB/s eta 0:00:03
   --------------- ------------------------ 1.6/4.0 MB 953.2 kB/s eta 0:00:03
   ------------------ --------------------- 1.8/4.0 MB 986.7 kB/s eta 0:00:03
   --------------------- ------------------ 2.1/4.0 MB 1.0 MB/s eta 0:00:02
   ----------------------- -

In [3]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
search.run("arsenal vs psg match summary")

c:\Users\niran\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


'LOKALAFDELINGER ARSENAL FC er en af Englands største og mest vindende fodboldklubber. En historie, der rækker mere end 100 år tilbage, har ført stribevis af … Arsenal Denmark gør opmærksom på, at der ikke er fortrydelsesret på indmeldelse i foreningen, og at betaling af kontingentet således ikke kan fortrydes eller kræves … Arsenals hjemmebanetrøje for sæsonen 2025/26 hylder klubbens stolte tradition med den klassiske røde overdel og de skinnende hvide ærmer. Logoet er holdt i rene … Tilmeldelse af Arsenal Denmarks nyhedsbrev med mange vigtige oplysninger, datoer og invitationer. Hvor mange lod du som medlem har forud for vores billettrækningsvindue i … Jan 8, 2024 · Transfervinduet er atter åbent, hvorfor vi i følgende artikel vil dykke ned i de mulige til- og afgange, vi kan se nærmere på for Mikel Artetas mandskab. Arsenal er efter …'

In [4]:
from langchain.tools import ShellTool

shell_tool = ShellTool()
result = shell_tool.run({"commands": ["echo 'Hello World!'", "time"]})

Executing command:
 ["echo 'Hello World!'", 'time']


c:\Users\niran\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [5]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama2", temperature=0)

C:\Users\niran\AppData\Local\Temp\ipykernel_23516\3666644574.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama2", temperature=0)


In [6]:
llm.invoke("how many letters in the word educa?")

AIMessage(content='The word "educá" has 8 letters.', additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2025-07-30T01:38:43.6900284Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 9433078700, 'load_duration': 5682700300, 'prompt_eval_count': 29, 'prompt_eval_duration': 2431197400, 'eval_count': 13, 'eval_duration': 1312037900}, id='run--f0010e1c-138c-45e2-becc-b96260d2654c-0')

In [9]:
from langchain.agents import tool

@tool
def get_word_length(word: str):
    """Returns the length of a word."""
    return len(word)

In [11]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a very powerful assistant but not great at calculating word lengths.",

        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [14]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.agents.format_scratchpad import format_to_tool_messages
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import tool
import json

# --- Define a simple tool ---
@tool
def get_weather(location: str) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is sunny."

tools = [get_weather]

# Create a string describing the tools for the prompt
tool_descs = "\n".join([f"{t.name}: {t.description}" for t in tools])

# --- Custom Output Parser ---
class SimpleToolOutputParser(AgentOutputParser):
    def parse(self, text: str):
        if "Action:" in text:
            try:
                action_name = text.split("Action:")[1].split("\n")[0].strip()
                action_input_str = text.split("Action Input:")[1].strip()
                try:
                    action_input = json.loads(action_input_str)
                except:
                    action_input = {"input": action_input_str}
                return AgentAction(tool=action_name, tool_input=action_input, log=text)
            except Exception as e:
                raise ValueError(f"Could not parse tool action: {text}") from e
        else:
            return AgentFinish(return_values={"output": text.strip()}, log=text)

# --- Prompt Template ---
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     f"You are a helpful AI assistant. You have access to the following tools:\n\n{tool_descs}\n\n"
     "When you want to use a tool, reply in the following format:\n"
     "Action: <tool name>\n"
     "Action Input: <JSON input>\n\n"
     "When you want to give the final answer, just reply with it directly."
    ),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# --- Initialize Ollama model ---
llm = ChatOllama(model="llama2", temperature=0)

# --- Agent chain ---
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_tool_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm
    | SimpleToolOutputParser()
)

# --- Test the agent ---
result = agent.invoke({
    "input": "What's the weather in New York?",
    "intermediate_steps": []
})

print(result)


tool='get_weather' tool_input={'location': 'New York'} log='Action: get_weather\nAction Input: { "location": "New York" }'


In [19]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a very powerful assistant but not great at calculating word lengths.",
            
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [20]:
from langchain.schema.messages import HumanMessage, AIMessage

chat_history = []



In [21]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

NameError: name 'llm_with_tools' is not defined

In [23]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend([
    HumanMessage(content=input1),
    AIMessage(content=result["Output"]),
])
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})

NameError: name 'agent_executor' is not defined

In [26]:
!pip install --upgrade google-api-python-client


   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
    --------------------------------------- 0.3/13.7 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.7 MB 2.7 MB/s eta 0:00:05
   ---- ----------------------------------- 1.6/13.7 MB 2.6 MB/s eta 0:00:05
   ------ --------------------------------- 2.1/13.7 MB 2.7 MB/s eta 0:00:05
   ------- -------------------------------- 2.6/13.7 MB 2.7 MB/s eta 0:00:05
   --------- ------------------------------ 3.1/13.7 MB 2.7 MB/s eta 0:00:04
   ----------- ---------------------------- 3.9/13.7 MB 2.8 MB/s eta 0:00:04
   ------------- -------------------------- 4.5/13.7 MB 2.8 MB/s eta 0:00:04
   -------------- ------------------------- 5.0/13.7 MB 2.8 MB/s eta 0:00:04
   ---------------- ----------------------- 5.8/13.7 MB 2.8 MB/s eta 0:00:03
   ------------------ --------------------- 6.3/13.7 MB 2.8 MB/s eta 0:00:03
   ------------------- -------------------- 6.8/13.7 MB 2.8 MB/s eta 0:00:03
   ----------

C:\Users\niran\AppData\Roaming\Python\Python310\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
C:\Users\niran\AppData\Roaming\Python\Python310\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
C:\Users\niran\AppData\Roaming\Python\Python310\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [27]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.7)
synopsis_template = "You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.\n\nTitle: {title}\nEra: {era}\nPlaywright: This is a synopsis for the above play:"
synopsis_prompt_template = PromptTemplate(input_variables=["title", "era"], template=synopsis_template)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt_template, output_keys="synopsis")

review_template = "You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.\n\nPlay Synopsis:\n{synopsis}\nReview from a New York Times play critic of the above play:"
prompt_template = PromptTemplate(inpput_variables = ["synopsis"], template=review_template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    output_variables=["synopsis", "review"],
    verbose=True
)

overall_chain({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})



C:\Users\niran\AppData\Local\Temp\ipykernel_23516\2298691558.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7)


ValidationError: 1 validation error for OpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0.7, 'mod...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [29]:
from langchain.memory import SimpleMemory
from langchain.chains import SequentialChain

template = "You are a social media manager for a theater company. Given the title of play, the era it is set in, the date, time and location, the synopsis of the play, and the review of the play,\
    it is your job to write a social media post for that play.\n\nHere is some context about the time and location of the play:\nDate and Time: {time}\nLocation: {location}\n\nPlay Synopsis:\n{synopsis}\nReview from a New York Times play critic of the above play:\n{review}\n\nSocial Media Post:"
    
prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=template)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(memories={"time": "December 25th, * pm PST", "location": "Theater in the Park"}),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    output_variables=["social_post_text"],
    verbose=True
)    
overall_chain({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})


NameError: name 'synopsis_chain' is not defined